### Leer el archivo JSON usando el "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
countries_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

In [0]:
countries_df = spark.read.schema(countries_schema).json(
    f"{bronze_folder_path}/{v_file_date}/country.json"
)

In [0]:
countries_dropped_df = countries_df.drop('countryIsoCode')

In [0]:
from pyspark.sql.functions import current_timestamp, lit


In [0]:
countries_final_df = countries_dropped_df.withColumnsRenamed({
    'countryId': 'country_id',
    'countryName': 'country_name',
}).withColumns({
    'ingestion_date': current_timestamp(),
    'enviroment': lit("production"),
    'file_date': lit(v_file_date)
})

In [0]:
#spark.sql("DROP TABLE IF EXISTS movie_silver.countries")

overwrite_partition(countries_final_df, "movie_silver", "countries", "file_date")


In [0]:
#countries_final_df.write.mode("overwrite").parquet("/mnt/datamoviehistory/silver/countries")
#countries_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.countries")
countries_final_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.countries")

In [0]:
%fs
ls /mnt/datamoviehistory/silver/countries

In [0]:
%sql
select file_date, count(1)
from movie_silver.countries
group by file_date;